Vi ønsket å estimere CO₂-utslipp fra lakseeksport. Dataene fra SSB inneholder hverken informasjon om transportmiddel eller destinasjon. For å kunne utføre ulike miljødataanalyser med våre data som utgangspunkt, er vi derfor nødt til å gjøre realistiske antagelser. 
For å kunne gjøre dette antar vi at fersk laks fraktes med fly (1.1 kg CO₂ per tonn-km, 2000 km) og frosset laks med skip (0.01 kg CO₂ per tonn-km, 5000 km). Disse estimatene gir et grovt men illustrativt bilde av transportrelaterte utslipp. Dette estimatet er ikke 100% presist, men gir en god indikasjon på miljøpåvirkningen over tid. Ved å gjøre disse realistiske antagelsene kan vi analysere trender og utvikling.

In [7]:
import requests
import pandas as pd
from itertools import product
from pandasql import sqldf # pip install pandasql

# ------------------------------
# 1. Hente data fra SSB
# ------------------------------
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")


# ------------------------------
# 2. Prosessere og strukturere data
# ------------------------------
def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values
    
    # Pivot for å få én kolonne per måling
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()

    # Navngi kolonner tydelig
    df_pivot.columns = ["År og ukenr.", 
                        "Fersk laks - Kilospris", 
                        "Fersk laks - Vekt (tonn)", 
                        "Frosset laks - Kilospris", 
                        "Frosset laks - Vekt (tonn)"]

    return df_pivot


# ------------------------------
# 3. Beregne CO₂-utslipp
# ------------------------------
CO2_FACTORS = {
    "Fersk laks": 1.1,    # kg CO₂ per tonn-km (fly)
    "Frosset laks": 0.01  # kg CO₂ per tonn-km (skip)
}

DISTANCES = {
    "Fersk laks": 2000,
    "Frosset laks": 5000
}

def beregn_co2_utslipp(df):
    """Legger til CO₂-utslipp basert på vekt og transporttype."""
    # Smelt datasettet for lettere håndtering
    df_melted = pd.melt(
        df, 
        id_vars=["År og ukenr."], 
        value_vars=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"],
        var_name="Type", 
        value_name="Vekt (tonn)"
    )
    df_melted.dropna(subset=["Vekt (tonn)"], inplace=True)

    # Trekk ut transporttype
    df_melted["Transporttype"] = df_melted["Type"].apply(lambda x: "Fersk laks" if "Fersk" in x else "Frosset laks")

    # Beregn CO₂-utslipp med list comprehension
    df_melted["CO2-utslipp (kg)"] = [
        vekt * CO2_FACTORS[typ] * DISTANCES[typ]
        for vekt, typ in zip(df_melted["Vekt (tonn)"], df_melted["Transporttype"])
    ]
    
    return df_melted


# ------------------------------
# 4. Bruk av Pandas SQL for analyse
# ------------------------------
def total_utslipp_per_uke(df):
    """Oppsummerer CO₂-utslipp per uke med SQL."""
    query = """
        SELECT 
            `År og ukenr.`, 
            Transporttype, 
            SUM(`CO2-utslipp (kg)`) AS `Total CO2-utslipp (kg)`
        FROM df
        GROUP BY `År og ukenr.`, Transporttype
        ORDER BY `År og ukenr.`
    """
    return sqldf(query)


# ------------------------------
# 5. Kjør alt sammen
# ------------------------------
def main():
    try:
        # Hent og prosesser data
        data = fetch_data()
        df_pivot = process_data(data)

        # Rens: fjern rader med NaN i vekt, som er nullverdier
        df_clean = df_pivot.dropna(subset=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"], how='all')

        # Definerer CO₂ utslipp 
        df_with_co2 = beregn_co2_utslipp(df_clean) 

        # defienrer sammenfattet data
        df_summary = total_utslipp_per_uke(df_with_co2)

        return df_summary

    except Exception as e:
        print("Feil:", e)
        return None


df_summary = main()
print(df_summary)

#if __name__ == "__main__":
    #main()
    



     År og ukenr. Transporttype  Total CO2-utslipp (kg)
0         2000U01    Fersk laks               8201600.0
1         2000U01  Frosset laks                 19150.0
2         2000U02    Fersk laks               8918800.0
3         2000U02  Frosset laks                 10800.0
4         2000U03    Fersk laks               8894600.0
...           ...           ...                     ...
2643      2025U18  Frosset laks                 23950.0
2644      2025U19    Fersk laks              39760600.0
2645      2025U19  Frosset laks                 26500.0
2646      2025U20    Fersk laks              42950600.0
2647      2025U20  Frosset laks                 24900.0

[2648 rows x 3 columns]


In [8]:
import requests
import pandas as pd
from itertools import product
from pandasql import sqldf # pip install pandasql

# ------------------------------
# 1. Hente data fra SSB
# ------------------------------
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    """Henter data fra SSB API og returnerer JSON-responsen."""
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    response = requests.post(api_url, json=query)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Feil ved henting av data: {response.status_code}\n{response.text}")


# ------------------------------
# 2. Prosessere og strukturere data
# ------------------------------
def process_data(data):
    """Prosesserer JSON-data og returnerer en formatert Pandas DataFrame."""
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values
    
    # Pivot for å få én kolonne per måling
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"], values="Value", aggfunc="sum").reset_index()

    # Navngi kolonner tydelig
    df_pivot.columns = ["År og ukenr.", 
                        "Fersk laks - Kilospris", 
                        "Fersk laks - Vekt (tonn)", 
                        "Frosset laks - Kilospris", 
                        "Frosset laks - Vekt (tonn)"]

    return df_pivot


# ------------------------------
# 3. Beregne CO₂-utslipp
# ------------------------------
CO2_FACTORS = {
    "Fersk laks": 1.1,    # kg CO₂ per tonn-km (fly)
    "Frosset laks": 0.01  # kg CO₂ per tonn-km (skip)
}

DISTANCES = {
    "Fersk laks": 2000,
    "Frosset laks": 5000
}

def beregn_co2_utslipp(df):
    """Legger til CO₂-utslipp basert på vekt og transporttype."""
    # Smelt datasettet for lettere håndtering
    df_melted = pd.melt(
        df, 
        id_vars=["År og ukenr."], 
        value_vars=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"],
        var_name="Type", 
        value_name="Vekt (tonn)"
    )
    df_melted.dropna(subset=["Vekt (tonn)"], inplace=True)

    # Trekk ut transporttype
    df_melted["Transporttype"] = df_melted["Type"].apply(lambda x: "Fersk laks" if "Fersk" in x else "Frosset laks")

    # Beregn CO₂-utslipp med list comprehension
    df_melted["CO2-utslipp (kg)"] = [
        vekt * CO2_FACTORS[typ] * DISTANCES[typ]
        for vekt, typ in zip(df_melted["Vekt (tonn)"], df_melted["Transporttype"])
    ]
    
    return df_melted


# ------------------------------
# 4. Bruk av Pandas SQL for analyse
# ------------------------------
def total_utslipp_per_uke(df):
    """Oppsummerer CO₂-utslipp per uke med SQL."""
    query = """
        SELECT 
            `År og ukenr.`, 
            Transporttype, 
            SUM(`CO2-utslipp (kg)`) AS `Total CO2-utslipp (kg)`
        FROM df
        GROUP BY `År og ukenr.`, Transporttype
        ORDER BY `År og ukenr.`
    """
    return sqldf(query)


# ------------------------------
# 5. Kjør alt sammen
# ------------------------------
def main():
    try:
        # Hent og prosesser data
        data = fetch_data()
        df_pivot = process_data(data)

        # Rens: fjern rader med NaN i vekt, som er nullverdier
        df_clean = df_pivot.dropna(subset=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"], how='all')

        # Definerer CO₂ utslipp 
        df_with_co2 = beregn_co2_utslipp(df_clean) 

        # defienrer sammenfattet data
        df_summary = total_utslipp_per_uke(df_with_co2)

        return df_summary

    except Exception as e:
        print("Feil:", e)
        return None


df_summary = main()

import plotly.graph_objects as go

# Konverter "År og ukenr." til ukeformat
df_summary["Uke"] = df_summary["År og ukenr."].str.replace("U", "-")

# Filtrer kun fersk laks (om du bare vil sammenligne den)
fersk = df_summary[df_summary["Transporttype"] == "Fersk laks"].copy()

# Flett inn vekten fra pivot-tabellen
df_pivot = process_data(fetch_data())  # Hent pivot på nytt
fersk_vekter = df_pivot[["År og ukenr.", "Fersk laks - Vekt (tonn)"]]
fersk = pd.merge(fersk, fersk_vekter, on="År og ukenr.", how="left")

# Plot
fig = go.Figure()

# CO₂-utslipp
fig.add_trace(go.Scatter(
    x=fersk["Uke"],
    y=fersk["Total CO2-utslipp (kg)"],
    name="CO₂-utslipp (kg)",
    mode="lines+markers",
    yaxis="y1"
))

# Vekt
fig.add_trace(go.Scatter(
    x=fersk["Uke"],
    y=fersk["Fersk laks - Vekt (tonn)"],
    name="Fersk laks – Vekt (tonn)",
    mode="lines+markers",
    yaxis="y2"
))

# Layout
fig.update_layout(
    title="CO₂-utslipp og vekt av fersk laks per uke",
    xaxis=dict(title="Uke"),
    yaxis=dict(title="CO₂-utslipp (kg)", side="left"),
    yaxis2=dict(title="Vekt (tonn)", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=600
)

#fig.show() #grafen er synlig lenger ned



print(df_summary)

#if __name__ == "__main__":
    #main()
    
""" 
    Koden viser en tabel som oppsummerer og viser estimert CO₂-utslipp (i kg) per uke, fordelt på type av laks, ferk eller frossen.
    Formålet med denne tabellen er å kunne analysere hvordan utslippene utvikler seg over tid og sammenligne miljøpåvirkningen.
"""


     År og ukenr. Transporttype  Total CO2-utslipp (kg)      Uke
0         2000U01    Fersk laks               8201600.0  2000-01
1         2000U01  Frosset laks                 19150.0  2000-01
2         2000U02    Fersk laks               8918800.0  2000-02
3         2000U02  Frosset laks                 10800.0  2000-02
4         2000U03    Fersk laks               8894600.0  2000-03
...           ...           ...                     ...      ...
2643      2025U18  Frosset laks                 23950.0  2025-18
2644      2025U19    Fersk laks              39760600.0  2025-19
2645      2025U19  Frosset laks                 26500.0  2025-19
2646      2025U20    Fersk laks              42950600.0  2025-20
2647      2025U20  Frosset laks                 24900.0  2025-20

[2648 rows x 4 columns]


' \n    Koden viser en tabel som oppsummerer og viser estimert CO₂-utslipp (i kg) per uke, fordelt på type av laks, ferk eller frossen.\n    Formålet med denne tabellen er å kunne analysere hvordan utslippene utvikler seg over tid og sammenligne miljøpåvirkningen.\n'

In [9]:
"""
    Denne koden visualiserer CO₂-utslipp og eksportert mengde for fersk laks per uke over tid.
    Grafen bruker to y-akser for å vise hvordan utslippene øker samtidig med produksjonen: én for vekt (tonn) og én for CO₂-utslipp (kg).
    Denne visualiseringen er nyttig for å analysere miljøpåvirkningen av lakseeksport og for å identifisere perioder med veldig høyt karbonavtrykk. 
    Grafen viser også sesongvariasjoner og kan brukes til videre vurdering av bærekraftige tiltak.
"""


import requests
import pandas as pd
from itertools import product
from pandasql import sqldf
import plotly.graph_objects as go

# ------------------------------------------
# 1. Hent data fra SSB
# ------------------------------------------
API_URL = "https://data.ssb.no/api/v0/no/table/03024/"

def fetch_data(api_url=API_URL):
    query = {"query": [], "response": {"format": "json-stat2"}}
    response = requests.post(api_url, json=query)
    response.raise_for_status()
    return response.json()

# ------------------------------------------
# 2. Prosesser og pivotér data
# ------------------------------------------
def process_data(data):
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))
    
    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values
    
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"],
                              values="Value", aggfunc="sum").reset_index()
    
    df_pivot.columns = ["År og ukenr.", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    
    return df_pivot

# ------------------------------------------
# 3. Beregn CO₂-utslipp
# ------------------------------------------
CO2_FACTORS = {
    "Fersk laks": 1.1,     # kg CO₂ per tonn-km
    "Frosset laks": 0.01   # kg CO₂ per tonn-km
}
DISTANCES = {
    "Fersk laks": 2000,    # km
    "Frosset laks": 5000
}

def beregn_co2(df):
    df_melted = pd.melt(df,
        id_vars=["År og ukenr."],
        value_vars=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"],
        var_name="Type", value_name="Vekt (tonn)"
    )
    df_melted.dropna(subset=["Vekt (tonn)"], inplace=True)
    df_melted["Transporttype"] = df_melted["Type"].apply(lambda x: "Fersk laks" if "Fersk" in x else "Frosset laks")
    
    df_melted["CO2-utslipp (kg)"] = [
        vekt * CO2_FACTORS[typ] * DISTANCES[typ]
        for vekt, typ in zip(df_melted["Vekt (tonn)"], df_melted["Transporttype"])
    ]
    return df_melted

# ------------------------------------------
# 4. Oppsummer CO₂ per uke
# ------------------------------------------
def total_utslipp_per_uke(df):
    query = """
    SELECT `År og ukenr.`, Transporttype,
           SUM(`CO2-utslipp (kg)`) AS `Total CO2-utslipp (kg)`
    FROM df
    GROUP BY `År og ukenr.`, Transporttype
    ORDER BY `År og ukenr.`
    """
    return sqldf(query)

# ------------------------------------------
# 5. Kjør alt
# ------------------------------------------
def main():
    data = fetch_data()
    df_pivot = process_data(data)
    df_clean = df_pivot.dropna(subset=["Fersk laks - Vekt (tonn)", "Frosset laks - Vekt (tonn)"], how="all")
    
    df_with_co2 = beregn_co2(df_clean)
    df_summary = total_utslipp_per_uke(df_with_co2)
    
    # Filtrer for fersk laks
    fersk = df_summary[df_summary["Transporttype"] == "Fersk laks"].copy()
    fersk["Uke"] = fersk["År og ukenr."].str.replace("U", "-")
    
    # Hent vekt for fersk laks
    fersk_vekter = df_pivot[["År og ukenr.", "Fersk laks - Vekt (tonn)"]]
    fersk = pd.merge(fersk, fersk_vekter, on="År og ukenr.", how="left")
    
    return fersk

# ------------------------------------------
# 6. Visualisering
# ------------------------------------------
df_fersk = main()

# Visualiser CO₂ og vekt tydelig

import plotly.graph_objects as go

fig = go.Figure()

# CO₂-utslipp (venstre y-akse)
fig.add_trace(go.Scatter(
    x=df_fersk["Uke"],
    y=df_fersk["Total CO2-utslipp (kg)"],
    name="CO₂-utslipp (kg)",
    mode="lines+markers",
    line=dict(color="blue", width=2),
    yaxis="y1"
))

# Fersk laks – vekt (høyre y-akse)
fig.add_trace(go.Scatter(
    x=df_fersk["Uke"],
    y=df_fersk["Fersk laks - Vekt (tonn)"],
    name="Fersk laks – Vekt (tonn)",
    mode="lines+markers",
    line=dict(color="red", width=2, dash="solid"),
    yaxis="y2"
))


fig.update_layout(
    title="CO₂-utslipp og vekt av fersk laks per uke",
    xaxis=dict(title="Uke"),
    yaxis=dict(
        title=dict(text="CO₂-utslipp (kg)", font=dict(color="blue")),
        tickfont=dict(color="blue"),
        side="left"
    ),
    yaxis2=dict(
        title=dict(text="Vekt (tonn)", font=dict(color="red")),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right"
    ),
    legend=dict(x=0.01, y=0.99),
    template="plotly_white",
    height=650
)


fig.show()

